In [115]:
import pandas as pd
import numpy
# !pip install transitionMatrix

In [116]:
df_checkinpd=pd.read_csv('client270.csv')
df_checkinpd.dropna()

len_whole=len(df_checkinpd)
df_checkinpd[['date','time']] = df_checkinpd.timestamp.str.split(" ",expand=True)
df_checkinpd[['time_1','time_2','time_3']] = df_checkinpd.time.str.split(":",expand=True)
df_checkinpd.drop(["time_3"], axis=1, inplace=True)

In [139]:
df_checkinpd.loc[df_checkinpd['building']=='Bldg23']

,Unnamed: 0,timestamp,client_df_2,AP,idx,client1,building,date,time,time_1,time_2
601,601,2014-01-11 08:36:08,4bffe1add69fc0e499e6ad46168530388532a038,Bldg23AP30,1137346,client694,Bldg23,2014-01-11,08:36:08,08,36
10308,10308,2014-04-28 20:18:55,4bffe1add69fc0e499e6ad46168530388532a038,Bldg23AP31,111669443634,client694,Bldg23,2014-04-28,20:18:55,20,18
10312,10312,2014-04-28 20:26:45,4bffe1add69fc0e499e6ad46168530388532a038,Bldg23AP30,111669444707,client694,Bldg23,2014-04-28,20:26:45,20,26
10345,10345,2014-04-28 22:25:16,4bffe1add69fc0e499e6ad46168530388532a038,Bldg23AP31,111669458066,client694,Bldg23,2014-04-28,22:25:16,22,25
10347,10347,2014-04-28 22:26:19,4bffe1add69fc0e499e6ad46168530388532a038,Bldg23AP26,111669458166,client694,Bldg23,2014-04-28,22:26:19,22,26


In [118]:
def warn(*args, **kwargs):
    pass
    import warnings
    warnings.warn = warn

In [119]:
list_date=list(df_checkinpd.date.unique())
dfArray = [df_checkinpd.loc[df_checkinpd.date == x] for x in list_date]

In [120]:
def get_set(df_1):
    set1=[df_1.index[0]]
    index_end=0
    for index, row in df_1.iterrows():
        index_start=index
        for index_2 in range(index_start,df_1.index[-1]+1):
            if (row['building']!=df_1['building'][index_2]):
                index_end=index_2
                set1.append(index_2)
                break
    set1.append(df_1.index[-1])
    return set1

In [121]:
def get_list(df_1,list1):
    len2=len(list1)
    list2=[]
    time_period=0
    for i in range(len2-1):
        if list1[i+1]-list1[i]==1: time_period=0;
        else: time_period=(int(df_1['time_1'][list1[i+1]-1])*60+int(df_1['time_2'][list1[i+1]-1]))-(int(df_1['time_1'][list1[i]])*60+int(df_1['time_2'][list1[i]]))
        list2.append(time_period)
    return list2

In [122]:
list_1,list_2,list_3,list_4=[],[],[],[]
for i in range(len(list_date)):
    list_1=list(set(get_set(dfArray[i])))
    list_1.sort()
    list_4=list_4+list_1[:len(list_1)-1]
    list_2=get_list(dfArray[i],list_1)
    list_3=list_3+list_2
df_state=df_checkinpd.loc[list_4,:]
df_state['stay time']=list_3
df_state.drop(['client_df_2','AP','idx','client1'],axis=1,inplace=True) 
df_state_1=df_state.loc[df_state['stay time']>=30] 


In [123]:
# df_state_1['date1']=pd.to_datetime(df_state_1['date'],infer_datetime_format=True)
df_state_1['date_1']= pd.to_datetime(df_state_1['date']).dt.dayofweek
df_state_1['date_day of week']=df_state_1['date_1'].map({0: 'Monday', 1: 'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'})
len_state1=len(df_state_1)

C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [124]:
s=list(df_state_1['time_1'])
len_first=len(s)
list2=[]
for list1 in s:
    if int(list1)<12: list2.append('Morning')
    if (int(list1)>=12) & (int(list1)<19): list2.append('afternoon')
    if int(list1)>=19: list2.append('Evening')
len_last=len(list2)
if len_first==len_last: print("correct conversion")

correct conversion


In [125]:
df_state_1['each_day']=list2
df_state_1['observation']=df_state_1['date_day of week']+'_'+df_state_1.each_day
df_markov=df_state_1
df_markov=df_markov[['building','stay time','observation']]
df_markov.dropna()

list_ob=df_markov.observation.unique()
list_building=df_markov.building.unique()

# list_building = list_building[~pd.isnull(list_building)]
# list_ob = list_ob[~pd.isnull(list_ob)]
df_markov=df_markov.reset_index(drop=True)

C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [126]:
df_state_1=df_state_1.reset_index(drop=True)
df_check=df_state_1
df_markov=df_state_1

In [127]:
list_datafrme=[]
def even(df_markov,building):
    list_time=[]
    list_time_1=[]
    df_label=df_markov[df_markov['building']==building]
    sum1=df_label['stay time'].sum()
    list_time=list(df_label['stay time'])
    for i in list_time:
        list_time_1.append(round(i/sum1,2))
    df_label['prob']=list_time_1
    return df_label
for building in list_building: 
    df_even=even(df_markov,building)
    list_datafrme.append(df_even)
df_markov_1=pd.concat(list_datafrme)

C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [128]:
df_markov_1['combine']=df_markov_1.observation+'/'+df_markov_1.building
list_combine=df_markov_1['combine'].unique()


In [129]:
result={}
def even(df_markov,combine):
    dict1={}
    df_label=df_markov[df_markov['combine']==combine]
    sum1=df_label['prob'].sum()
    dict1[combine]=sum1
    return dict1
for combine in list_combine: 
    result.update(even(df_markov_1,combine))
df_markov_2=pd.DataFrame(list(result.items()), columns=['combine', 'prob'])
df_markov_2[['observation','hidden']] = df_markov_2['combine'].str.split("/",expand=True)
df_markov_2=df_markov_2.drop(columns='combine')

In [130]:
markov_emission=pd.DataFrame(columns=list_ob,index=list_building)
for index,row in markov_emission.iterrows():
    for index1,row1 in df_markov_2.iterrows():
         markov_emission[row1['observation']][row1['hidden']]=row1['prob']

In [131]:
list_building

array(['Bldg44', 'Bldg48', 'Bldg3', 'Bldg4', 'Bldg30_tmp_', 'Bldg15',
       'Bldg8'], dtype=object)

In [132]:
markov_emission


,Wednesday_Morning,Thursday_afternoon,Friday_afternoon,Saturday_afternoon,Sunday_afternoon,Sunday_Evening,Monday_afternoon,Tuesday_Morning,Wednesday_afternoon,Wednesday_Evening,...,Friday_Morning,Monday_Morning,Monday_Evening,Tuesday_Evening,Saturday_Evening,Sunday_Morning,Tuesday_afternoon,Thursday_Evening,Saturday_Morning,Friday_Evening
Bldg44,0.02,0.12,0.07,0.07,0.05,0.01,0.07,0.06,0.06,0.01,...,0.05,0.04,0.01,0,0,0.02,0.02,0.04,0.01,0.01
Bldg48,0.06,0.1,0.05,0.04,0.05,0,0.07,0,0.12,0,...,0.11,0.05,0,0.01,0.02,0.02,0.11,0,0.02,0
Bldg3,NaN,0.34,0.18,NaN,NaN,NaN,NaN,NaN,0.28,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,NaN
Bldg4,NaN,0.04,NaN,NaN,0.42,NaN,NaN,NaN,0.2,NaN,...,NaN,NaN,0.06,NaN,0.05,0.15,NaN,NaN,0.09,NaN
Bldg30_tmp_,NaN,0.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.38,NaN,NaN
Bldg15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bldg8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [133]:
markov_emission.to_csv(r'C:\Users\dell\Documents\bia678\markov chain\markov_emission_client17.csv')
df_markov.to_csv(r'C:\Users\dell\Documents\bia678\markov chain\markov_client170.csv')

In [134]:
df_markov.building.unique()

array(['Bldg44', 'Bldg48', 'Bldg3', 'Bldg4', 'Bldg30_tmp_', 'Bldg15',
       'Bldg8'], dtype=object)

In [135]:

df_markov_2

,prob,observation,hidden
0,0.02,Wednesday_Morning,Bldg44
1,0.12,Thursday_afternoon,Bldg44
2,0.07,Friday_afternoon,Bldg44
3,0.07,Saturday_afternoon,Bldg44
4,0.05,Sunday_afternoon,Bldg44
5,0.01,Sunday_Evening,Bldg44
6,0.06,Tuesday_Morning,Bldg44
7,0.06,Wednesday_afternoon,Bldg44
8,0.01,Wednesday_Evening,Bldg44
9,0.04,Thursday_Morning,Bldg44


In [136]:
df_markov['building'].unique()

array(['Bldg44', 'Bldg48', 'Bldg3', 'Bldg4', 'Bldg30_tmp_', 'Bldg15',
       'Bldg8'], dtype=object)